In [ ]:
!pip install -q pyomo

In [ ]:
from pyomo.environ import * 

In [ ]:
import numpy as np

optimization problem:\
max 35x1 + 57x2 + 48x3 + 20x4 + 15x5 \
s.t : \
 x1 + x2 + x3 <= 75 \
x3 + x4 + x5 <= 100 \
x1 + x2 >= 10 \
30000x1 + 95000x2 + 10000x3 + 5000x4 + 4000x5 <= 1500000\
100000x1 + 500000x2 + 200000x3 + 45000x4 + 25000x5 >= 900000\
$\ x_{i}\ $>=0 $\forall\text{ i}\text{ }\epsilon\text{ }\text{{1,2,3,4,5}} $

$\textbf{1.ANSWER:}$ \
\begin{align}
\ z_{1} = x_{1}+x_{2}\ , 
\ z_{2} = x_{3}+x_{4}+x_{5}\\\
\ |z_{1} - z_{2}|<= 18,\\\
\ i.e.\text{ }|x_{1}+x_{2} - (x_{3}+x_{4}+x_{5})| <=18\ 
\end{align}
\begin{align}
\text{ }The\text{ }\ constraint \text{ }\ “|x_{1}+x_{2} -(x_{3}+x_{4}+x_{5})| ≤ 18\ ” \
\text{is not a linear constraint. }\
\end{align}
$\textbf{Approach 1:}$\
However, the constraint can be transformed into linear constraints using a simple “technique”.The constraint  
“$|x_{1}+x_{2} - (x_{3}+x_{4}+x_{5})| ≤ 18 $” is equivalent to the following two constraints: \
$x_{1}+x_{2} - (x_{3}+x_{4}+x_{5}) ≤ 18 $ and \
$-x_{1}-x_{2} + x_{3}+x_{4}+x_{5} ≤ 18 $ \
If we replace the original constraint by these two constraints, we obtain a linear program.






In [ ]:
coef = np.loadtxt('lab6_Ex2_coef.txt', delimiter=',')

In [ ]:
print(coef.shape)
print('Num rows:',coef.shape[0])
print('Num cols:',coef.shape[1])

print(coef)

(8, 6)
Num rows: 8
Num cols: 6
[[ 3.5e+01  5.7e+01  4.8e+01  2.0e+01  1.5e+01  0.0e+00]
 [ 1.0e+00  1.0e+00  1.0e+00  0.0e+00  0.0e+00  7.5e+01]
 [ 0.0e+00  0.0e+00  1.0e+00  1.0e+00  1.0e+00  1.0e+02]
 [-1.0e+00 -1.0e+00  0.0e+00  0.0e+00  0.0e+00 -1.0e+01]
 [ 3.0e+04  9.5e+04  1.0e+04  5.0e+03  4.0e+03  1.5e+06]
 [-1.0e+05 -5.0e+05 -2.0e+05 -4.5e+04 -2.5e+04 -9.0e+05]
 [ 1.0e+00  1.0e+00 -1.0e+00 -1.0e+00 -1.0e+00  1.8e+01]
 [-1.0e+00 -1.0e+00  1.0e+00  1.0e+00  1.0e+00  1.8e+01]]


In [ ]:
model = ConcreteModel()

In [ ]:
N = coef.shape[1]-1
M = coef.shape[0]-1
print(N,M)

5 7


In [ ]:
obj_coef = coef[0,:-1]
print(obj_coef.shape)
print(obj_coef)

(5,)
[35. 57. 48. 20. 15.]


In [ ]:
constr_coef = coef[1:,:-1]
print(constr_coef.shape)
print(constr_coef)

(7, 5)
[[ 1.0e+00  1.0e+00  1.0e+00  0.0e+00  0.0e+00]
 [ 0.0e+00  0.0e+00  1.0e+00  1.0e+00  1.0e+00]
 [-1.0e+00 -1.0e+00  0.0e+00  0.0e+00  0.0e+00]
 [ 3.0e+04  9.5e+04  1.0e+04  5.0e+03  4.0e+03]
 [-1.0e+05 -5.0e+05 -2.0e+05 -4.5e+04 -2.5e+04]
 [ 1.0e+00  1.0e+00 -1.0e+00 -1.0e+00 -1.0e+00]
 [-1.0e+00 -1.0e+00  1.0e+00  1.0e+00  1.0e+00]]


In [ ]:
constr_rhs = coef[1:,-1]
print(constr_rhs.shape)
print(constr_rhs)

(7,)
[ 7.5e+01  1.0e+02 -1.0e+01  1.5e+06 -9.0e+05  1.8e+01  1.8e+01]


In [ ]:
col_indices = np.arange(N)
print(col_indices)

model.x = Var(col_indices, domain=NonNegativeIntegers)

row_indices = np.arange(M)
print(row_indices)


[0 1 2 3 4]
[0 1 2 3 4 5 6]


In [ ]:
model.constraints = ConstraintList()

In [ ]:
for i in row_indices:
  model.constraints.add(summation(constr_coef[i][0:N],model.x)  <= constr_rhs[i])

In [ ]:
model.objective = Objective(expr = summation(obj_coef[0:N],model.x), sense=maximize)

In [ ]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    7 : {1, 2, 3, 4, 5, 6, 7}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

1 Var Declarations
    x : Size=5, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeIntegers
          1 :     0 :  None :  None : False :  True : NonNegativeIntegers
          2 :     0 :  None :  None : False :  True : NonNegativeIntegers
          3 :     0 :  None :  None : False :  True : NonNegativeIntegers
          4 :     0 :  None :  None : False :  True : NonNegativeIntegers

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 35.0*x[0] + 57.0*x[1] + 4

In [ ]:
!apt-get install -y -qq coinor-cbc

In [ ]:
opt_cbc = SolverFactory('cbc')

In [ ]:
result = opt_cbc.solve(model)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)


Problem: 
- Name: unknown
  Lower bound: 3378.0
  Upper bound: 3378.0
  Number of objectives: 1
  Number of constraints: 7
  Number of variables: 5
  Number of binary variables: 0
  Number of integer variables: 5
  Number of nonzeros: 5
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.01
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 1
  Error rc: 0
  Time: 0.025562524795532227
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal


In [ ]:
# display solution
print('\nObjective = ', model.objective())

print('\nDecision Variables')
for i in range(N):
  print('x[',i,'] = ', model.x[i].value)
print('\n Constraints')
model.constraints.display()


Objective =  3378.0

Decision Variables
x[ 0 ] =  34.0
x[ 1 ] =  0.0
x[ 2 ] =  41.0
x[ 3 ] =  11.0
x[ 4 ] =  0.0

 Constraints
constraints : Size=7
    Key : Lower : Body        : Upper
      1 :  None :        75.0 :      75.0
      2 :  None :        52.0 :     100.0
      3 :  None :       -34.0 :     -10.0
      4 :  None :   1485000.0 : 1500000.0
      5 :  None : -12095000.0 : -900000.0
      6 :  None :       -18.0 :      18.0
      7 :  None :        18.0 :      18.0


$\textbf{Approach 2:}$\
Let:\
 $|x_{1}+x_{2} - (x_{3}+x_{4}+x_{5})| = a_{1}+a_{2} ,$
 and \
 $x_{1}+x_{2} - (x_{3}+x_{4}+x_{5})| = a_{1}-a_{2}$ \
with these assumptions our new optimization problem becomes linear with the following additional constraints : \
$x_{1}+x_{2} - x_{3}-x_{4}-x_{5} - a_{1} - a_{2} <= 0,$\
$-x_{1}-x_{2} + x_{3}+x_{4}+x_{5} - a_{1} - a_{2} <= 0,$\
$x_{1}+x_{2} - x_{3}-x_{4}-x_{5} - a_{1} + a_{2} <= 0,$\
$-x_{1}-x_{2} + x_{3}+x_{4}+x_{5} + a_{1} - a_{2} <= 0,$\
$ \ a_{1} + a_{2} <= 18, $\
where $a_{i} >= 0$







In [ ]:
new_coef = np.loadtxt('lab6_Ex2_coef_2.txt', delimiter=',')

In [ ]:
print(new_coef.shape)
print('Num rows:',new_coef.shape[0])
print('Num cols:',new_coef.shape[1])

(11, 8)
Num rows: 11
Num cols: 8


In [ ]:
print(new_coef)

[[ 3.5e+01  5.7e+01  4.8e+01  2.0e+01  1.5e+01  0.0e+00  0.0e+00  0.0e+00]
 [ 1.0e+00  1.0e+00  1.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00  7.5e+01]
 [ 0.0e+00  0.0e+00  1.0e+00  1.0e+00  1.0e+00  0.0e+00  0.0e+00  1.0e+02]
 [-1.0e+00 -1.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00 -1.0e+01]
 [ 3.0e+04  9.5e+04  1.0e+04  5.0e+03  4.0e+03  0.0e+00  0.0e+00  1.5e+06]
 [-1.0e+05 -5.0e+05 -2.0e+05 -4.5e+04 -2.5e+04  0.0e+00  0.0e+00 -9.0e+05]
 [ 1.0e+00  1.0e+00 -1.0e+00 -1.0e+00 -1.0e+00 -1.0e+00 -1.0e+00  0.0e+00]
 [-1.0e+00 -1.0e+00  1.0e+00  1.0e+00  1.0e+00 -1.0e+00 -1.0e+00  0.0e+00]
 [ 1.0e+00  1.0e+00 -1.0e+00 -1.0e+00 -1.0e+00 -1.0e+00  1.0e+00  0.0e+00]
 [-1.0e+00 -1.0e+00  1.0e+00  1.0e+00  1.0e+00  1.0e+00 -1.0e+00  0.0e+00]
 [ 0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00  1.0e+00  1.0e+00  1.8e+01]]


In [ ]:
new_model = ConcreteModel()

In [ ]:
M = new_coef.shape[0]-1
N = new_coef.shape[1]-1
print(M,N)

10 7


In [ ]:
new_obj_coef = new_coef[0,:-1]
print(new_obj_coef.shape)
print(new_obj_coef)

(7,)
[35. 57. 48. 20. 15.  0.  0.]


In [ ]:
new_constr_coef = new_coef[1:,:-1]
print(new_constr_coef.shape)
print(new_constr_coef)

(10, 7)
[[ 1.0e+00  1.0e+00  1.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00]
 [ 0.0e+00  0.0e+00  1.0e+00  1.0e+00  1.0e+00  0.0e+00  0.0e+00]
 [-1.0e+00 -1.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00]
 [ 3.0e+04  9.5e+04  1.0e+04  5.0e+03  4.0e+03  0.0e+00  0.0e+00]
 [-1.0e+05 -5.0e+05 -2.0e+05 -4.5e+04 -2.5e+04  0.0e+00  0.0e+00]
 [ 1.0e+00  1.0e+00 -1.0e+00 -1.0e+00 -1.0e+00 -1.0e+00 -1.0e+00]
 [-1.0e+00 -1.0e+00  1.0e+00  1.0e+00  1.0e+00 -1.0e+00 -1.0e+00]
 [ 1.0e+00  1.0e+00 -1.0e+00 -1.0e+00 -1.0e+00 -1.0e+00  1.0e+00]
 [-1.0e+00 -1.0e+00  1.0e+00  1.0e+00  1.0e+00  1.0e+00 -1.0e+00]
 [ 0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00  1.0e+00  1.0e+00]]


In [ ]:
new_constr_rhs = new_coef[1:,-1]
print(new_constr_rhs.shape)
print(new_constr_rhs)

(10,)
[ 7.5e+01  1.0e+02 -1.0e+01  1.5e+06 -9.0e+05  0.0e+00  0.0e+00  0.0e+00
  0.0e+00  1.8e+01]


In [ ]:
col_indices_x = np.arange(N-2)
print(col_indices_x)
new_model.x = Var(col_indices_x ,domain=NonNegativeIntegers)



[0 1 2 3 4]


In [ ]:
col_indices_a = np.arange(2)
print(col_indices_a)
new_model.a = Var(col_indices_a, domain=NonNegativeReals)

[0 1]


In [ ]:
row_indices = np.arange(M)
print(row_indices)

[0 1 2 3 4 5 6 7 8 9]


In [ ]:
new_model.constraints = ConstraintList()

In [ ]:
for i in row_indices:
  new_model.constraints.add(summation(new_constr_coef[i][0:N-2],new_model.x) + summation(new_constr_coef[i][N-2:N],new_model.a) <= new_constr_rhs[i])

In [ ]:
new_model.objective = Objective(expr = summation(new_obj_coef[0:N-2],new_model.x) + summation(new_obj_coef[N-2:N],new_model.a), sense=maximize)

In [ ]:
new_model.pprint()

3 Set Declarations
    a_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {0, 1}
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

2 Var Declarations
    a : Size=2, Index=a_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals
    x : Size=5, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeIntegers
          1 :     0 :  None :  None : False :  True : NonNegativeIntegers
          2 :     

In [ ]:
!apt-get install -y -qq coinor-cbc

In [ ]:
opt_cbc = SolverFactory('cbc')

In [ ]:
result = opt_cbc.solve(new_model)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)


Problem: 
- Name: unknown
  Lower bound: 3378.0
  Upper bound: 3378.0
  Number of objectives: 1
  Number of constraints: 10
  Number of variables: 7
  Number of binary variables: 0
  Number of integer variables: 5
  Number of nonzeros: 5
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.01
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 1
  Error rc: 0
  Time: 0.022630929946899414
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal


In [ ]:
# display solution
print('\nObjective = ', new_model.objective())

print('\nDecision Variables')
for i in range(N-2):
  print('x[',i,'] = ', new_model.x[i].value)
for j in range(2):
  print('a[',j,'] = ', new_model.a[j].value)
print('\n Constraints')
new_model.constraints.display()


Objective =  3378.0

Decision Variables
x[ 0 ] =  34.0
x[ 1 ] =  0.0
x[ 2 ] =  41.0
x[ 3 ] =  11.0
x[ 4 ] =  0.0
a[ 0 ] =  0.0
a[ 1 ] =  18.0

 Constraints
constraints : Size=10
    Key : Lower : Body        : Upper
      1 :  None :        75.0 :      75.0
      2 :  None :        52.0 :     100.0
      3 :  None :       -34.0 :     -10.0
      4 :  None :   1485000.0 : 1500000.0
      5 :  None : -12095000.0 : -900000.0
      6 :  None :       -36.0 :       0.0
      7 :  None :         0.0 :       0.0
      8 :  None :         0.0 :       0.0
      9 :  None :         0.0 :       0.0
     10 :  None :        18.0 :      18.0


$4.ANSWER: $ \
The solver status and solver termination condition in both approaches 1 and 2 is:\
$\text{Solver status: ok} $\
$\text{Solver termination condition: optimal}$ \
Yes, We are getting optimal solution in each case. \


$5.ANSWER:$ \
$\text{APPROACH 1:}$\
Objective =  3378.0

Decision Variables\
x[ 0 ] =  34.0\
x[ 1 ] =  0.0\
x[ 2 ] =  41.0\
x[ 3 ] =  11.0\
x[ 4 ] =  0.0 \

Constraints\
constraints : Size=7\
    Key : Lower : Body        : Upper\
      1 :  None :        75.0 :      75.0\
      2 :  None :        52.0 :     100.0\
      3 :  None :       -34.0 :     -10.0\
      4 :  None :   1485000.0 : 1500000.0\
      5 :  None : -12095000.0 : -900000.0\
      6 :  None :       -18.0 :      18.0\
      7 :  None :        18.0 :      18.0

$\text{APPROACH 2:}$ \

Objective =  3378.0 \

Decision Variables\
x[ 0 ] =  34.0\
x[ 1 ] =  0.0\
x[ 2 ] =  41.0\
x[ 3 ] =  11.0\
x[ 4 ] =  0.0\
a[ 0 ] =  0.0\
a[ 1 ] =  18.0 

 Constraints\
constraints : Size=10\
    Key : Lower : Body        : Upper\
      1 :  None :        75.0 :      75.0\
      2 :  None :        52.0 :     100.0\
      3 :  None :       -34.0 :     -10.0\
      4 :  None :   1485000.0 : 1500000.0\
      5 :  None : -12095000.0 : -900000.0\
      6 :  None :       -36.0 :       0.0\
      7 :  None :         0.0 :       0.0\
      8 :  None :         0.0 :       0.0\
      9 :  None :         0.0 :       0.0\
     10 :  None :        18.0 :      18.0

$6.ANSWER :$ \
$\text{APPROACH 1:}$ \
Optimal values of the Original Decision Variables\
x[ 0 ] = 34.0,
x[ 1 ] = 0.0,
x[ 2 ] = 41.0,
x[ 3 ] = 11.0,
x[ 4 ] = 0.0 \
$\text{APPROACH 2:}$ \
x[ 0 ] = 34.0,
x[ 1 ] = 0.0,
x[ 2 ] = 41.0,
x[ 3 ] = 11.0,
x[ 4 ] = 0.0 \

$Observation:$
$\text{Here in both approaches 1 and 2 the optimal values of the original decision variables are same}$


$\textbf{7.ANSWER:}$ \
\begin{align}
\ z_{3} = x_{4}+x_{5}\ , 
\ |z_{3} - x_{3}|>= 4,\\\
\ i.e.\text{ }|x_{4}+x_{5} - x_{3}| >= 4\ 
\end{align}
\begin{align}
\text{ }The\text{ }\ constraint \text{ }\ “|x_{4}+x_{5} -x_{3}| >= 4 \ ” \
\text{is not a linear constraint. }\
\end{align}
$\textbf{Approach 1:}$\
However, the constraint can be transformed into linear constraints using a substitution: \\
$|x_{4}+x_{5} - x_{3}| = p $ \\
With this substitution, note that we have  $\ p >= 0$ 

\begin{align}
\ p >= x_{4}+x_{5}-x_{3}\text{ or }\ -x_{4}-x_{5}+x_{3}-p<=0, \\
\ p >= -x_{4}-x_{5}+x_{3}\text{ or }\ x_{4}+x_{5}-x_{3}-p<=0, \\
\ p >= 4 \\
\end{align}

If we replace the original constraint by these three constraints, we obtain a linear program.\

$\textbf{Approach 2:}$ \
$Let:$\
$|x_{4}+x_{5} - x_{3}| = a_{1}+a_{2} ,$
 and \
 $x_{4}+x_{5} - x_{3} = a_{1}-a_{2}$ \
with these assumptions our new optimization problem becomes linear with the following additional constraints : \
$x_{4}+x_{5} - x_{3} - a_{1} - a_{2} <= 0,$\
$-x_{4}-x_{5} + x_{3} - a_{1} - a_{2} <= 0,$\
$x_{4}+x_{5} - x_{3} - a_{1} + a_{2} <= 0,$\
$-x_{4}-x_{5} + x_{3} + a_{1} - a_{2} <= 0,$\
$ \ a_{1} + a_{2} >= 4, $\
where $a_{i} >= 0$
